In [37]:
import pandas as pd

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import GOOG

In [2]:

btc = pd.read_csv("../data/BTCUSDT.csv")
btc.drop_duplicates(inplace=True)
btc["time_open"] = pd.to_datetime(btc.time_open)
btc.set_index("time_open", inplace=True)
btc = btc[::-1].copy()

btc_bt = pd.DataFrame(index=btc.index.copy())
btc_bt["Open"] = btc.open
btc_bt["High"] = btc.high
btc_bt["Low"] = btc.low
btc_bt["Close"] = btc.close
btc_bt["Volume"] = btc.volume
btc_bt

,Open,High,Low,Close,Volume
time_open,,,,,
2017-08-17 07:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
2017-08-17 07:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2017-08-17 07:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
2017-08-17 07:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
2017-08-17 07:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...
2024-07-13 18:10:00,58767.16,58803.25,58764.00,58803.25,7.688330
2024-07-13 18:11:00,58803.25,58803.25,58786.05,58790.00,11.794740
2024-07-13 18:12:00,58789.99,58795.82,58781.14,58795.82,5.881740


In [3]:
class MyStrategy(Strategy):
    def init(self):
        self.sma8 = self.I(lambda series: pd.Series(series).rolling(8).mean(), self.data.Close)
        self.sma16 = self.I(lambda series: pd.Series(series).rolling(16).mean(), self.data.Close)
    
    def next(self):
        if crossover(self.sma8, self.sma16):
            self.position.close()
        elif crossover(self.sma16, self.sma8):
            self.buy()

In [38]:
bt = Backtest(GOOG, MyStrategy, cash=1_000_000)

stats = bt.run()
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    42.73743
Equity Final [$]                     926316.0
Equity Peak [$]                    1505325.29
Return [%]                            -7.3684
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   -0.893934
Volatility (Ann.) [%]               23.441388
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -60.331757
Avg. Drawdown [%]                  -12.488026
Max. Drawdown Duration     1892 days 00:00:00
Avg. Drawdown Duration      296 days 00:00:00
# Trades                                   66
Win Rate [%]                        56.060606
Best Trade [%]                      15.236155
Worst Trade [%]                    -23.114855
Avg. Trade [%]                    

In [39]:
bt.plot()

ValueError: failed to validate DatetimeTickFormatter(id='p1046', ...).days: expected a value of type str, got ['%d %b', '%a %d'] of type list

In [12]:
btc_bt.sort_index(inplace=True)

In [32]:
all_timestamps = pd.date_range(start=btc_bt.index.min(), end=btc_bt.index.max(), freq='T')
missing_timestamps = all_timestamps.difference(btc_bt.index)
missing_timestamps

/var/folders/z1/0tkms1m53hb8x5dkfgl_g8800000gn/T/ipykernel_40546/814063501.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  all_timestamps = pd.date_range(start=btc_bt.index.min(), end=btc_bt.index.max(), freq='T')


DatetimeIndex(['2017-08-17 07:05:00', '2017-08-17 07:06:00',
               '2017-08-17 07:07:00', '2017-08-17 07:08:00',
               '2017-08-17 07:09:00', '2017-08-17 07:10:00',
               '2017-08-17 07:11:00', '2017-08-17 07:12:00',
               '2017-08-17 07:13:00', '2017-08-17 07:14:00',
               ...
               '2024-03-31 03:50:00', '2024-03-31 03:51:00',
               '2024-03-31 03:52:00', '2024-03-31 03:53:00',
               '2024-03-31 03:54:00', '2024-03-31 03:55:00',
               '2024-03-31 03:56:00', '2024-03-31 03:57:00',
               '2024-03-31 03:58:00', '2024-03-31 03:59:00'],
              dtype='datetime64[ns]', length=24691, freq=None)

In [14]:
len(missing_timestamps)

24691

In [28]:
btc_bt_resambled = btc_bt.resample('min').asfreq()
btc_bt_resambled.ffill(inplace=True)

/var/folders/z1/0tkms1m53hb8x5dkfgl_g8800000gn/T/ipykernel_40546/1921863723.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  btc_bt_resambled = btc_bt.resample('T').asfreq()


In [33]:
btc_bt.index[btc_bt.index.duplicated()]

DatetimeIndex([], dtype='datetime64[ns]', name='time_open', freq='min')

In [25]:
btc_bt = btc_bt.groupby(btc_bt.index).mean()

In [27]:
btc_bt

,Open,High,Low,Close,Volume
time_open,,,,,
2017-08-17 07:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
2017-08-17 07:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2017-08-17 07:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
2017-08-17 07:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
2017-08-17 07:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...
2024-07-13 18:10:00,58767.16,58803.25,58764.00,58803.25,7.688330
2024-07-13 18:11:00,58803.25,58803.25,58786.05,58790.00,11.794740
2024-07-13 18:12:00,58789.99,58795.82,58781.14,58795.82,5.881740


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3608081 entries, 2017-08-17 07:00:00 to 2024-07-13 18:14:00
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Open    float64
 1   High    float64
 2   Low     float64
 3   Close   float64
 4   Volume  float64
dtypes: float64(5)
memory usage: 294.2 MB
